In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling3D

C:\Users\simir\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
import talos as ta
import wrangle as wr
from talos.metrics.keras_metrics import fmeasure_acc
from talos import live

import pandas as pd

from keras.models import Sequential
from keras.layers import Dropout, Dense

# Keras items
from keras.optimizers import Adam, Nadam
from keras.activations import relu, elu
from keras.losses import categorical_crossentropy

In [7]:
def load_images(folder):
    images = [loadmat(os.path.join(folder, filename))['X'] for filename in os.listdir(folder) if filename.endswith('.mat')]
    classification = [loadmat(os.path.join(folder, filename))['TICI_report'] for filename in os.listdir(folder) if filename.endswith('.mat')]
    return images, classification

In [8]:
def dsa_classifier_model(x_train, y_train, x_test, y_test, params):
    train_d_gen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)
    
    train_d_gen.fit(x_train)
    
    train_generator = train_d_gen.flow(x_train, y_train, batch_size)
    
    validation_generator = ImageDataGenerator().flow(x_test, y_test)
    
    base_model = InceptionV3(weights='imagenet', include_top=False)

    # DNN portion
    # average pooling layer
    x = GlobalAveragePooling3D()(base_model.output)
    # fully-connected layer
    x = Dense(1024, activation=params['activation'])(x)
    # logistic layer
    predictions = Dense(5, activation=params['last_activation'])(x) # 5 is the number of classes in the data, since TICI is 0,1,2a,2b,3

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(loss=params['losses'],
                  optimizer=params['optimizer'](),
                  metrics=['acc', fmeasure_acc])
    
    history = model.fit_generator(
        train_generator,
        validation_data=validation_generator,
        batch_size=params['batch_size'],
        callbacks=[live()],
        epochs=params['epochs'])

    return history, model

In [9]:
p = {'hidden_layers':[0, 1, 2],
     'batch_size': [20, 30],
     'epochs': [10, 50, 100],
     'optimizer': [Nadam, Adam],
     'losses': [categorical_crossentropy],
     'activation':[relu, elu],
     'last_activation': ['softmax']}

In [ ]:
images, classification = load_images(folder)
t = ta.Scan(x=images,
            y=classification,
            model=dsa_classifier_model,
            params=p,
            dataset_name='stroke_dataset',
            experiment_no='1')